In [96]:
import math
import csv
import random
import numpy as np
from scipy import optimize

In [97]:
with open('mydatabase.csv') as f:
    reader = csv.reader(f)
    my_data = list(reader)

my_data[0][0]='Food'

In [98]:
with open('foodcosts.csv') as f:
    reader = csv.reader(f)
    my_costs = list(reader)

#Cost per 100g of food item
food_costs=np.array([i[3] for i in my_costs[1::]],dtype=float)

In [99]:
commodities = [i[0] for i in my_data[1::]]
print(commodities)

['White, self raising flour', 'Brown, wholemeal flour', 'Bread, white sliced pan', 'Bread, brown sliced pan', 'Spaghett', 'Sirloin steak', 'Roast beef - topside or rib', 'Sliced / diced beef pieces', 'Pork loin chops', 'Pork steak', 'Lamb - whole leg / half leg', 'Lamb loin chops', 'Chicken', "Lamb's liver", 'Ham fillet', 'Cooked ham ', 'Best back rashers', 'Pork sausages', 'Fresh fillet of cod', 'Fresh salmon', 'Fresh hake', 'Smoked salmon ', 'Full fat milk ', 'Low fat milk ', 'Irish cheddar', 'Large eggs', 'Butter', 'Grapes ', 'Bananas', 'Tomatoes', 'Onions', 'Broccoli', 'Carrots ', 'Mushrooms', 'Tomatoes tinned', 'Potatoes', 'White granulated sugar', 'Jam ', 'Marmalade', 'Tea bags', 'Orange juice']


In [100]:
nutrients = my_data[0]
print(nutrients)

['Food', 'Energy (mcal)', 'Carbs (g)', 'Protein (g)', 'Fat (g)', 'Fibre (g)', 'Potassium (mg)', 'Calcium (mg)', 'Magnesium (mg)', 'Phosphorus (mg)', 'Iron (mg)', 'Copper (mg)', 'Zinc (mg)', 'Manganese (mg)', 'Selenium (ug)', 'Iodine (ug)', 'Vitamin D (ug)', 'Vitamin K (ug)', 'Thiamin (mg)', 'Riboflavin (mg)', 'Niacin (mg)', 'Vitamin B6 (mg)', 'Folate (ug) ', 'Biotin (ug)', 'Vitamin C (mg) ', 'Vitamin A (ug)', 'Vitamin E (mg)', 'Pantothenic acid (mg)', 'Vitamin B12 (ug)']


In [143]:
#Recommended daily intake of Male 70kg 18years + 
rda_man = [2.7, 351, 58.1, 84, 25, 3500, 950, 350, 550, 11, 1.6, 16.3, 3, 
           70, 150, 15, 70, 0.1, 1.6, 1.6, 1.7, 330, 40, 110, 750, 13, 5, 4]

In [154]:
def perform_simplex(data, b_vector):
    #Cost of each food will just be sum of x vector so c=1
    c=np.ones(len(data))

    #Matrix A of vitamins for each food
    A = np.transpose(np.matrix(data,dtype=float)) 

    #Recommended daily vitamin allowance for a moderately active man
    b = np.array(b_vector)

    #Minimisation problem with lower bound
    solution = optimize.linprog(c, A_ub=-A, b_ub=-b, method='simplex')
    
    return solution

In [198]:
def optimise_diet(nutrient_index):

    rda=[rda_man[i] for i in nutrient_index]

    #Nutrient composition of food from database
    data = [[i[j+1] for j in nutrient_index] for i in my_data[1::]]

    print("Nutrients to optimise: \n",[my_data[0][j+1] for j in nutrient_index])
    print("\n")

    optimum_ans = perform_simplex(data,rda)
    print(optimum_ans)

    #Print out the relevant foods
    print("\nShopping List:\n")
    for i in np.nonzero(optimum_ans.x)[0]:
        print(round(100*optimum_ans.x[i]/food_costs[i],2),"g ",
              commodities[i],"= €",round(optimum_ans.x[i],6))
    

    print("\nTotal daily cost = €",round(sum(optimum_ans.x),2))
    print("\nTotal yearly cost = €",round(sum(optimum_ans.x*365.25),2))

    print("\nNutrient intake (%) with this diet:")
    print("------------------------------")


    excess = []
    for j in range(1,len(nutrients)):
        nut = np.array([i[j] for i in my_data[1::]],dtype=float)
        xs=round(np.matmul(nut,optimum_ans.x),1)
        m=rda_man[j-1]
        xsp = round(100*abs(xs/m),0)
        excess.append(xsp)
        
        print(nutrients[j],xsp)

    print("\nNutrients Not Satisfied:")
    print([[i,nutrients[i+1],excess[i]] for i, x in enumerate(excess) if x < 100])

In [199]:
#First just try Calories, Carbs, Protein, and Fat

basic_allowance=[0,1,2,3]

optimise_diet(basic_allowance)

Nutrients to optimise: 
 ['Energy (mcal)', 'Carbs (g)', 'Protein (g)', 'Fat (g)']


     fun: 1.3031837936412036
 message: 'Optimization terminated successfully.'
     nit: 6
   slack: array([   0.        ,  104.90167962,    0.        ,    0.        ])
  status: 0
 success: True
       x: array([ 0.64759256,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.07228783,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.5833034 ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ])

Shopping List:

572.08 g  White, self raising flour = € 0.647593
25.31 g  Chicken = € 0.072288
87.9 g  Butter = € 0.583

In [200]:
#Optimise over most nutrients
most_nutrients_idx = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,19,20,21,22,23,24,25,26,27]

optimise_diet(most_nutrients_idx)


Nutrients to optimise: 
 ['Energy (mcal)', 'Carbs (g)', 'Protein (g)', 'Fat (g)', 'Fibre (g)', 'Potassium (mg)', 'Calcium (mg)', 'Magnesium (mg)', 'Phosphorus (mg)', 'Iron (mg)', 'Copper (mg)', 'Zinc (mg)', 'Manganese (mg)', 'Selenium (ug)', 'Iodine (ug)', 'Vitamin D (ug)', 'Vitamin K (ug)', 'Niacin (mg)', 'Vitamin B6 (mg)', 'Folate (ug) ', 'Biotin (ug)', 'Vitamin C (mg) ', 'Vitamin A (ug)', 'Vitamin E (mg)', 'Pantothenic acid (mg)', 'Vitamin B12 (ug)']


     fun: 5.05724855147625
 message: 'Optimization terminated successfully.'
     nit: 137
   slack: array([  4.67012053e-02,   4.22670358e+01,   7.02042855e+01,
         0.00000000e+00,   5.43776765e+01,   1.26255465e+03,
         0.00000000e+00,   2.04157872e+02,   1.98861123e+03,
         1.33270197e+01,   8.81327558e-01,   0.00000000e+00,
         7.04879408e+00,   7.78931136e+01,   1.39520723e+02,
         0.00000000e+00,   0.00000000e+00,   2.39520196e+01,
         1.60784924e+00,   8.63445551e+01,   7.72835671e+01,
         0.0

In [400]:
def make_meals(recipes,commodities):
    
    A_sparse = np.zeros((len(recipes),len(commodities)))
 
    for i in range(A_sparse.shape[0]):
        for j in range(A_sparse.shape[1]):
            A_sparse[i,j] = random.Random(i*j).choice(amounts)
    
    #€ / Meal = Xg x € / 100g
    meal_costs = np.matmul(A_sparse,food_costs)

    #Nutrient / 100 g = € / 100g x Nutrient / €
    nutrients_per_100g = np.zeros(data.shape)
    for i,d in enumerate(data):
        nutrients_per_100g[i] = data[i]*food_costs[i]
    
    # Meal Nutrients = Xg x Nutrient / 100g
    meal_nutrients = np.matmul(A_sparse,nutrients_per_100g)

    # Meal Nutrients / Meal €
    meal_data = np.zeros(meal_nutrients.shape)
    
    for i,x in enumerate(meal_nutrients):
        meal_data[i] = meal_nutrients[i]/meal_costs[i]
    
    return meal_data, meal_nutrients, A_sparse

In [403]:
def optimise_meals(meal_nutrient_index):
    rda=[rda_man[i] for i in meal_nutrient_index]

    #Nutrient composition of food from database
    data = np.matrix([i[1::] for i in my_data[1::]],dtype=float)

    recipes = ['Meal 1', 'Meal 2', 'Meal 3', 'Meal 4', 'Meal 5',
            'Meal 6', 'Meal 7', 'Meal 8', 'Meal 9', 'Meal 10']
    
    #Amounts is in 100g quantities 
    amounts = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.1,0.2,0.5,1.0]

    meal_data, meal_nutrients, A_sparse = make_meals(recipes, commodities)

    meal_nutrients_red = np.matrix([[meal_nutrients[i,j] for j in meal_nutrient_index] for i in range(len(recipes))])
    
    print("Nutrients to optimise: \n",[my_data[0][j+1] for j in meal_nutrient_index])
    print("\n")

    meal_ans = perform_simplex(meal_nutrients_red, rda)
    print(meal_ans)

    #Print out the relevant foods
    print("\nShopping List:\n")
    for i in np.nonzero(meal_ans.x)[0]:
        print(round(100*meal_ans.x[i]/food_costs[i],2),"g ",
                recipes[i],"= €",round(meal_ans.x[i],6))

    print("\nTotal daily cost = €",round(sum(meal_ans.x),2))
    print("\nTotal yearly cost = €",round(sum(meal_ans.x*365.25),2))

    print("\nNutrient intake (%) with this diet:")
    print("------------------------------")

    excess = []
    for j in range(len(nutrients)-1):

        test = [meal_nutrients[i,j] for i,x in enumerate(meal_nutrients)]
        xs=round(np.matmul(test,meal_ans.x),1)
        m=rda_man[j]
        xsp = round(100*abs(xs/m),0)
        
        excess.append(xsp)
    
        print(nutrients[j+1],xsp)

    print("\nNutrients Not Satisfied:")
    print([[i,nutrients[i+1],excess[i]] for i, x in enumerate(excess) if x<100])
    
    return A_sparse

In [404]:
A = optimise_meals(basic_allowance)

Nutrients to optimise: 
 ['Energy (mcal)', 'Carbs (g)', 'Protein (g)', 'Fat (g)']


     fun: 3.51964045709915
 message: 'Optimization terminated successfully.'
     nit: 9
   slack: array([  7.78437095e-02,   0.00000000e+00,   1.18257250e+02,
         0.00000000e+00])
  status: 0
 success: True
       x: array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        2.47171815,  0.        ,  1.0479223 ,  0.        ,  0.        ])

Shopping List:

172.61 g  Meal 6 = € 2.471718
98.69 g  Meal 8 = € 1.047922

Total daily cost = € 3.52

Total yearly cost = € 1285.55

Nutrient intake (%) with this diet:
------------------------------
Energy (mcal) 104.0
Carbs (g) 100.0
Protein (g) 304.0
Fat (g) 100.0
Fibre (g) 134.0
Potassium (mg) 105.0
Calcium (mg) 62.0
Magnesium (mg) 120.0
Phosphorus (mg) 427.0
Iron (mg) 122.0
Copper (mg) 100.0
Zinc (mg) 102.0
Manganese (mg) 210.0
Selenium (ug) 171.0
Iodine (ug) 101.0
Vitamin D (ug) 169.0
Vitamin K (ug) 10.0
Thiamin (mg) 3300.0
Riboflavin

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:24: RuntimeWarning: invalid value encountered in true_divide


In [405]:
some_nutrients_idx = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,24,25]#,19,20,21,22,23,24,25,26,27]

meal_nutrients = optimise_meals(some_nutrients_idx)

Nutrients to optimise: 
 ['Energy (mcal)', 'Carbs (g)', 'Protein (g)', 'Fat (g)', 'Fibre (g)', 'Potassium (mg)', 'Calcium (mg)', 'Magnesium (mg)', 'Phosphorus (mg)', 'Iron (mg)', 'Copper (mg)', 'Zinc (mg)', 'Manganese (mg)', 'Selenium (ug)', 'Iodine (ug)', 'Vitamin D (ug)', 'Vitamin A (ug)', 'Vitamin E (mg)']


     fun: 5.3959204354738279
 message: 'Optimization terminated successfully.'
     nit: 30
   slack: array([  1.57760338e+00,   1.75087435e+02,   2.04197449e+02,
         5.54919783e+01,   4.01843816e+01,   2.23910035e+03,
         1.33627565e+02,   3.63757611e+02,   3.10349961e+03,
         1.32930142e+01,   1.18149660e+00,   1.15870266e+01,
         9.35037406e+00,   8.71514769e+01,   0.00000000e+00,
         2.09545551e+01,   0.00000000e+00,   6.64193154e+00])
  status: 0
 success: True
       x: array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        4.84064258,  0.        ,  0.55527786,  0.        ,  0.        ])

Shopping List:

338.03 g  Meal 6 = 

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:24: RuntimeWarning: invalid value encountered in true_divide


In [431]:
for j in range(A.shape[0]):
    print("\nMeal",j)
    [print([commodities[i],float(A[j][i])]) for i,x in enumerate(A[j]) if x>0]


Meal 0

Meal 1
['Sirloin steak', 1.0]
['Roast beef - topside or rib', 0.5]
['Lamb - whole leg / half leg', 0.5]
['Pork sausages', 0.1]
['Tomatoes', 0.2]
['Onions', 0.2]
['Mushrooms', 0.5]
['Tomatoes tinned', 0.1]
['Potatoes', 0.2]
['Jam ', 1.0]

Meal 2
['Bread, brown sliced pan', 0.5]
['Sirloin steak', 0.5]
['Cooked ham ', 0.2]
['Pork sausages', 0.1]
['Irish cheddar', 0.2]
['Bananas', 0.2]
['Tomatoes', 0.5]
['Broccoli', 0.5]
['Jam ', 1.0]

Meal 3
['Bread, white sliced pan', 0.5]
['Lamb - whole leg / half leg', 0.2]
['Lamb loin chops', 0.5]
['Best back rashers', 0.2]
['Grapes ', 0.1]
['Potatoes', 0.5]
['Orange juice', 0.1]

Meal 4
['Chicken', 0.2]
['Ham fillet', 0.2]
['Tomatoes', 1.0]
['Onions', 0.1]
['Tomatoes tinned', 0.5]

Meal 5
['Brown, wholemeal flour', 1.0]
['Bread, white sliced pan', 0.5]
['Roast beef - topside or rib', 0.2]
['Sliced / diced beef pieces', 0.2]
['Fresh salmon', 0.2]
['Smoked salmon ', 0.5]
['Irish cheddar', 0.1]
['Butter', 0.1]
['Broccoli', 0.5]
['Tea bags', 1.0